In [1]:
#3.9 标准化，让运营数据落入相同的区间
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [2]:
data=np.loadtxt('/Users/yezhibin/Downloads/书/数据分析书/python_book/chapter3/data6.txt',delimiter='\t')

In [5]:
#z-score 标准化
zscore-scaler=preprocessing.StandardScaler() 
data_scale_1=zscore_scaler.fit_transform(data)

SyntaxError: can't assign to operator (<ipython-input-5-1e6ce4530c48>, line 2)

In [6]:
#3.10 离散化，对连续运营数据做逻辑分层
import pandas as pd
from sklearn.cluster import KMeans
from sklearn import preprocessing

In [7]:
#读取数据
df=pd.read_table('/Users/yezhibin/Downloads/书/数据分析书/python_book/chapter3/data7.txt',names=['id','amount','income','datetime','age']) #读取数据文件
print(df.head(5))

      id  amount  income             datetime    age
0  15093    1390   10.40  2017-04-30 19:24:13   0-10
1  15062    4024    4.68  2017-04-27 22:44:59  70-80
2  15028    6359    3.84  2017-04-27 10:07:55  40-50
3  15012    7759    3.70  2017-04-04 07:28:18  30-40
4  15021     331    4.25  2017-04-08 11:14:00  70-80


In [8]:
#针对时间数据的离散化
for i, signle_data in enumerate(df['datetime']):
    single_data_tmp=pd.to_datetime(signle_data)
    df['datetime'][i]=single_data_tmp.weekday()
    
print(df.head(5))

/anaconda3/envs/py3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


      id  amount  income datetime    age
0  15093    1390   10.40        6   0-10
1  15062    4024    4.68        3  70-80
2  15028    6359    3.84        3  40-50
3  15012    7759    3.70        1  30-40
4  15021     331    4.25        5  70-80


In [9]:
#针对多值离散数据的离散化
map_df = pd.DataFrame([['0-10', '0-40'], ['10-20', '0-40'], ['20-30', '0-40'], ['30-40', '0-40'], ['40-50', '40-80'],
                       ['50-60', '40-80'], ['60-70', '40-80'], ['70-80', '40-80'], ['80-90', '>80'], ['>90', '>80']],
                      columns=['age', 'age2'])
df_tmp=df.merge(map_df,left_on='age',right_on='age',how='inner')
df=df_tmp.drop('age',1)
print(df.head(5))

      id  amount  income datetime  age2
0  15093    1390   10.40        6  0-40
1  15064    7952    4.40        0  0-40
2  15080     503    5.72        5  0-40
3  15068    1668    3.19        5  0-40
4  15019    6710    3.20        0  0-40


In [10]:
#针对连续数据的离散化
#方法1：自定义分箱区间实现离散化
bins=[0,200,1000,5000,10000] #自定义区间边界
df['amount1']=pd.cut(df['amount'],bins)
print(df.head(5))

      id  amount  income datetime  age2        amount1
0  15093    1390   10.40        6  0-40   (1000, 5000]
1  15064    7952    4.40        0  0-40  (5000, 10000]
2  15080     503    5.72        5  0-40    (200, 1000]
3  15068    1668    3.19        5  0-40   (1000, 5000]
4  15019    6710    3.20        0  0-40  (5000, 10000]


In [16]:
#方法2 使用聚类法实现离散化
data=df['amount']
data_reshape=data.reshape((data.shape[0],1))
model_kmeans=KMeans(n_clusters=4,random_state=0)
keames_result=model_kmeans.fit_predict(data_reshape)
df['amount2']=keames_result
print(df.head(5))

      id  amount  income datetime  age2        amount1  amount2
0  15093    1390   10.40        6  0-40   (1000, 5000]        2
1  15064    7952    4.40        0  0-40  (5000, 10000]        1
2  15080     503    5.72        5  0-40    (200, 1000]        2
3  15068    1668    3.19        5  0-40   (1000, 5000]        2
4  15019    6710    3.20        0  0-40  (5000, 10000]        1


/anaconda3/envs/py3/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  app.launch_new_instance()


In [17]:
#方法3：使用4分位数实现离散化
df['amount3']=pd.qcut(df['amount'],4,labels=['bad','medium','good','awesome'])
df=df.drop('amount',1)
print(df.head(5))

      id  income datetime  age2        amount1  amount2  amount3
0  15093   10.40        6  0-40   (1000, 5000]        2      bad
1  15064    4.40        0  0-40  (5000, 10000]        1  awesome
2  15080    5.72        5  0-40    (200, 1000]        2      bad
3  15068    3.19        5  0-40   (1000, 5000]        2      bad
4  15019    3.20        0  0-40  (5000, 10000]        1  awesome


In [18]:
import pandas as pd
import jieba  # 结巴分词
from sklearn.feature_extraction.text import TfidfVectorizer  # 基于TF-IDF的词频转向量库


# 分词函数
def jieba_cut(string):
    word_list = []  # 建立空列表用于存储分词结果
    seg_list = jieba.cut(string)  # 精确模式分词
    for word in seg_list:  # 循环读取每个分词
        word_list.append(word)  # 分词追加到列表
    return word_list


# 读取自然语言文件
fn = open('/Users/yezhibin/Downloads/书/数据分析书/python_book/chapter3/text.txt')
string_lines = fn.readlines()
fn.close()

# 中文分词
seg_list = []  # 建立空列表，用于存储所有分词结果
for string_line in string_lines:  # 读取每行数据
    each_list = jieba_cut(string_line)  # 返回每行的分词结果
    seg_list.append(each_list)  # 分词结果添加到结果列表
for i in range(5):  # 打印输出第一行的前5条数据
    print (seg_list[1][i])

# word to vector
stop_words = [u'\n', u'/', u'“', u'”', u'的', u'，', u'和', u'是', u'随着', u'对于', u'对', u'等', u'能', u'都', u'。', u'、',
              u'中', u'与', u'在', u'其']  # 自定义要去除的无用词
vectorizer = TfidfVectorizer(stop_words=stop_words, tokenizer=jieba_cut)  # 创建词向量模型
X = vectorizer.fit_transform(string_lines)  # 将文本数据转换为向量空间模型
vector = vectorizer.get_feature_names()  # 获得词向量
vector_value = X.toarray()  # 获得词向量值
vector_pd = pd.DataFrame(vector_value, columns=vector)  # 创建用于展示的数据框
print (vector_pd.head(1))  # 打印输出第一条数据

ModuleNotFoundError: No module named 'jieba'

In [19]:
import pandas as pd
import jieba  # 结巴分词
from sklearn.feature_extraction.text import TfidfVectorizer  # 基于TF-IDF的词频转向量库

ModuleNotFoundError: No module named 'jieba'